![logos](media/logos.png)

# Practical Demonstration of New User-Requested Nav2 Features #


## Who Am I?

Hi, I'm [Steve Macenski](https://www.linkedin.com/in/steve-macenski-41a985101/). I do a great deal of different "robotics stuff" at [Samsung Research America](https://www.sra.samsung.com/) as the *Open-Source Robotics, Engineering Lead*. I consult with internal Samsung robotics teams on technology and business, work on mobile robot and perception research, and lead the ROS2 mobile robotics ecosystem. I sit on the [ROS2 TSC](https://docs.ros.org/en/foxy/Governance.html) and I am the leader of the Navigation Working Group. 

Before my current role, I was the leader of the robotics team at [Simbe Robotics](https://www.simberobotics.com/) and worked on NASA's Asteroid Redirect Robotic Mission and RESTORE-L (a  servicing mission to Landsat-7).

## Overview

Nav2 is the second generation of the ROS Navigation Stack. It contains a growing set of capabilities, algorithms, and features to build production and research applications. You can find more information about Nav2 at: https://navigation.ros.org.

In the following talk, we're going to discuss some of the more basic new features in Nav2 that can help you build your research or applications. This includes:
    - Basic Nav2 operation through Rviz2 using **NavigateToPose**
    - Basic Nav2 operation through a Python3 script using **NavigateThroughPoses**
    - Use of the Waypoint Follower and Task Executor plugins
    - Introduction to keepout zones and speed restricted zones

We will then put all of these together to show a basic autonomous robotics demo based on Nav2. We will be doing this in a simulated warehouse where robots are often deployed:

<img src="media/warehouse.png" alt="drawing" width="800"/>


*Taken from [here](https://github.com/aws-robotics/aws-robomaker-small-warehouse-world).

## Nav2 Overview

- Independent, modular servers
- Configurable navigation logic, dynamically loaded BT nodes, and per-task behavior selection
- A growing set of run-time reconfigurable algorithms and plugins
- Software quality, linting, and testing (86% test coverage)
- Documentation and a rich community with Slack and Working Group (Join us! [Sign up for the working group here](https://calendar.google.com/calendar/u/0/embed?src=agf3kajirket8khktupm9go748@group.calendar.google.com&ctz=America/Los_Angeles))



<img src="https://navigation.ros.org/_images/architectural_diagram.png" alt="drawing" width="800"/>


---

## Part 1) Let's Get Started!

While that's building, lets inspect the `nav2_rosdevday_2021` package a little... You'll find:

    - launch: Launch files for the simulation, nav2, rviz, and robot state publisher. It's a one-stop shop for bringup. Setting `use_sim_time:=False` and `use_simulator:=False` allows you to launch with hardware!
    - maps: Maps and costmap filter masks for navigation
    - params: parameter files for navigation
    - worlds: simulation world files containing the robot and warehouse models
    - scripts: basic autonomy scripts

Once your packages have built and are setup, running the command below should launch Nav2, the simulation, and Rviz2:

`ros2 launch nav2_rosdevday_2021 system_launch.py`

<img src="media/initial_view.png" alt="drawing" width="800"/>


Now in your terminal you're probably seeing a bunch of messages pass by, that's OK. We have now yet set our initial position, so the servers are unable to finish the lifecycle initialization to start safe navigation. This is their way of individually complaining until you tell the system roughly where the robot is starting at.

You can do this through Rviz2 using **`2D Pose Estimation`** at the robot's position in the world for a demonstration or testing. You can also do this programmatically for a real system which we can mock up via:

`ros2 topic pub /initialpose geometry_msgs/msg/PoseWithCovarianceStamped "{header: {stamp: {sec: 0}, frame_id: 'map'}, pose: {pose: {position: {x: 3.45, y: 2.15, z: 0.0}, orientation: {z: 1.0, w: 0.0}}}}"`

<img src="media/pose_set.png" alt="drawing" width="800"/>

You should now see the robot initialized at its position, the costmaps update in Rviz2, and the warnings should stop.

Now, simply use the **`Nav2 Goal`** button to request the robot to go to a specific goal to test it out!

In [1]:
from IPython.display import HTML

HTML("""
    <video alt="test" loop autoplay width=800>
        <source src="./media/first_test.mp4" type="video/mp4">
    </video>
""")

#### **Now you're navigating!** 

Play around, get a feel for it. If you find yourself in a bad state at any time during this tutorial, feel free to close out the application and relaunch with the instructions above.

---

## Part 2) Navigate Actions

There are three primary navigation actions the Nav2 stack exposes currently.

### Navigate To Pose

Lets demonstrate this using Rviz2 and the Nav2 plugin to walk through a notional **pick and place** autonomy task.

The NavigateToPose action is most suitable for point-to-point navigation requests or for other tasks that can be represented in a behavior tree with a boundary condition pose, such as dynamic object following.

NavigateToPose.action:

``` bash
#goal definition
geometry_msgs/PoseStamped pose
string behavior_tree
---
#result definition
std_msgs/Empty result
---
# feedback definition
geometry_msgs/PoseStamped current_pose
builtin_interfaces/Duration navigation_time
builtin_interfaces/Duration estimated_time_remaining
int16 number_of_recoveries
float32 distance_remaining
```

As you can see, the action's primary inputs are the `pose` you'd like the robot to navigate to and the (optional) `behavior_tree` to use. If none is specified, it uses the default behavior tree in the BT Navigator. During the action's execution, you'll get feedback with important information like the robot's pose, how much time has elapsed, the estimated time remaining, the distance remaining, and the number of recoveries executed while navigating to the goal. This information can be used to make good autonomy decisions or track progress.  

come up with demo around this and explain the rviz plugin along the way TODO

go to manual forklift, talk about feedback, notional get load from person or arm, go to empty space, notional unloaded by another person or arm

### Navigate Through Poses

Lets demonstrate this using a python3 script to mock up a basic **security** autonomy task.

The NavigateThroughPoses action is most suitable for pose-constrained navigation requests or for other tasks that can be represented in a behavior tree with a set of poses.

NavigateThroughPoses.action:

``` bash
#goal definition
geometry_msgs/PoseStamped[] poses
string behavior_tree
---
#result definition
std_msgs/Empty result
---
#feedback definition
geometry_msgs/PoseStamped current_pose
builtin_interfaces/Duration navigation_time
builtin_interfaces/Duration estimated_time_remaining
int16 number_of_recoveries
float32 distance_remaining
int16 number_of_poses_remaining
```

As you can see, the action's inputs are nearly identical to NavigateToPose, except now we take in a vector of `poses`. The feedback is similar as well, only containing the new `number_of_poses_remaining` field to track progress through the via-points.

come up with demo around this and explain the python script along the way

come up with demo around this and explain the python script along the way TODO

### Waypoint Following

Lets demonstrate this using a python3 script to mock up a basic **inspection** autonomy task.

In [ ]:
(explain action, feedback, what this is good for)

come up with demo around this and explain the task executors along the way

---

## Part 3) Costmap Filters

---

## Part 4) Putting It All Together

In [ ]:
(launch file with keepout, speed zones, in warehouse, doing a task autonomously)

video here

---

# Conclusion